In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# install from git
!if [ -e ./rubiks ]; then rm -rf ./rubiks; fi
!pip uninstall rubiks_rl -y --quiet
!git clone https://github.com/LongDangHoang/rubik_rl ./rubiks --quiet
!cd ./rubiks; pip install . --quiet; cd ..

# some packages
!pip install torchinfo wandb --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [19]:
import numpy as np
import random
import pythreejs as p3

from torchinfo import summary

from rubiks_rl.colors import Color
from rubiks_rl.rubik54 import Rubik54
from rubiks_rl.models import RubikModel
from rubiks_rl.world import *

from google.colab import output
from IPython.display import display

output.enable_custom_widget_manager()

In [23]:
# Define hyper parameters
max_depth_scramble = 20
num_cubes = 100
num_epochs = 100

In [21]:
# how far in the scramble do we want?
#### If we have the model learn on few steps, it may struggle as the number of steps increase?
#### best to have this as a parameter
# for each sample, find best action by evaluating on breadth-1
#### first, generate in a batch fashion the states to be evaluated
#### this will have shape (num_cubes * max_depth_scramble * 12, 54, 6)
#### evaluate the batch and add in the reward (-1 if state not solve, else 1)
#### then, retrieve the bootstrapped labels by taking the argmax over the reward
# train the model using this output labels (num_cubes * max_depth_scramble) (by chunking over shape of 12)
## we also apply weighting based on scramble distance to the average when taking the loss
## this forms one epoch. Should profile
## repeat until reach max_epoch

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RubikModel(num_blocks=3).to(device)

# count params
summary(model, input_size=(16, 54, 6))

Layer (type:depth-idx)                   Output Shape              Param #
RubikModel                               [16, 1]                   --
├─Flatten: 1-1                           [16, 324]                 --
├─Sequential: 1-2                        [16, 324]                 --
│    └─Block: 2-1                        [16, 324]                 --
│    │    └─BatchNorm1d: 3-1             [16, 324]                 648
│    │    └─Linear: 3-2                  [16, 324]                 105,300
│    │    └─ELU: 3-3                     [16, 324]                 --
│    │    └─BatchNorm1d: 3-4             [16, 324]                 (recursive)
│    │    └─MultiheadAttention: 3-5      [16, 324]                 421,200
│    │    └─ELU: 3-6                     [16, 324]                 --
│    └─Block: 2-2                        [16, 324]                 --
│    │    └─BatchNorm1d: 3-7             [16, 324]                 648
│    │    └─Linear: 3-8                  [16, 324]              

Policy iteration loop

In [ ]:
n